In [1]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from ipywidgets import interact
from ml_collections import ConfigDict
from models.ETD_KT_CM_JAX_Vectorised import *
from filters import resamplers
from filters.filter import ParticleFilter

jax.config.update("jax_platform_name", "cpu")

import metrics.ensemble as ens_metrics


Initialisation

In [30]:
signal_params = ConfigDict(KDV_params_2_SALT)
print(signal_params)

Advection_basis_name: constant
E: 10
Forcing_basis_name: none
P: 1
S: 0
c_0: 0
c_1: 1
c_2: 0.0
c_3: 2.0e-05
c_4: 0.0
dt: 0.001
equation_name: KdV
initial_condition: gaussian
method: Dealiased_SETDRK4
noise_magnitude: 0.01
nx: 256
tmax: 1
xmax: 1
xmin: 0



In [31]:
ensemble_params = ConfigDict(KDV_params_2_SALT)
print(ensemble_params)

Advection_basis_name: constant
E: 10
Forcing_basis_name: none
P: 1
S: 0
c_0: 0
c_1: 1
c_2: 0.0
c_3: 2.0e-05
c_4: 0.0
dt: 0.001
equation_name: KdV
initial_condition: gaussian
method: Dealiased_SETDRK4
noise_magnitude: 0.01
nx: 256
tmax: 1
xmax: 1
xmin: 0



Next, we specify a signal, by choosing a deterministic solver, 

In [32]:
signal_params.update(E=1,nx = 256,P=1,S=0,tmax=1)
ensemble_params.update(E=128,noise_magnitude=0.01,P=1,tmax=1)
n_filter_steps = 50

Now we continue to define a stochastic ensemble

Now we specify the models, by calling the class.

In [33]:
signal_model = ETD_KT_CM_JAX_Vectorised(signal_params)
ensemble_model = ETD_KT_CM_JAX_Vectorised(ensemble_params)

import numpy as np
initial_signal = initial_condition(signal_model.x, signal_params.E, signal_params.initial_condition)
initial_ensemble = initial_condition(ensemble_model.x, ensemble_params.E, ensemble_params.initial_condition)

available_resamplers = ", ".join(resamplers.keys())
print(available_resamplers)

multinomial, systematic, no_resampling, default


In [34]:
obs_frequency = 32
observation_noise = 1e-4
observation_locations_params = jnp.arange(0,len(signal_model.x),obs_frequency)
data_assimilation_freq = 16
n_filter_steps = int(signal_model.nmax//data_assimilation_freq)

pf_systematic = ParticleFilter(
    n_particles = ensemble_params.E,
    n_steps = data_assimilation_freq,
    n_dim = initial_signal.shape[-1],
    forward_model = ensemble_model,
    signal_model = signal_model,
    sigma = observation_noise,
    seed = 11,
    resampling='multinomial',
    observation_locations = np.arange(0,signal_model.x.shape[0],128),
)

I think that the last input is not the 

In [35]:
final, all = pf_systematic.run(initial_ensemble, initial_signal, n_filter_steps) #the final input is scan length? 

Put in the initial condition.

In [36]:
particles =jnp.concatenate([initial_ensemble[None,...], all[0]], axis=0)
signal = jnp.concatenate([initial_signal[None,...], all[1]], axis=0)
print(f"Particles Shape: {particles.shape} is (N_da_steps+1, N_particles, N_dim)")
print(f"Signal Shape: {signal.shape} is (N_da_steps+1, 1,  N_dim)")
print(len(all))
observations = all[2]
print(f"Observations Shape: {observations.shape} is (N_da_steps, 1,  N_dim)")

Particles Shape: (63, 128, 256) is (N_da_steps+1, N_particles, N_dim)
Signal Shape: (63, 1, 256) is (N_da_steps+1, 1,  N_dim)
3
Observations Shape: (62, 1, 256) is (N_da_steps, 1,  N_dim)


In [37]:
def plot(da_step):
    plt.plot(signal_model.x, signal[da_step,0,:], color='k',label='signal')
    plt.plot(signal_model.x, particles[da_step,:,:].T, color='b',label='particles',linewidth=0.1)
    plt.show()

interact(plot, da_step=(0, n_filter_steps))

interactive(children=(IntSlider(value=31, description='da_step', max=62), Output()), _dom_classes=('widget-int…

<function __main__.plot(da_step)>